In [1]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Chargement des données
df = pd.read_csv("movie_review.csv")

# Prétraitement des données textuelles
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalpha()]
    tokens = [token for token in tokens if token not in stop_words]
    return tokens

df['text'] = df['text'].apply(preprocess_text)




j ai rencontre une erreur a ce niv la et j ai pas trouve de solution

In [2]:
# Entraînement du modèle Word2Vec
model = Word2Vec(df['text'], vector_size=100, window=5, min_count=1, workers=4)

# Vectorisation des reviews de movies
def vectorize_text(text):
    vectors = [model.wv[word] for word in text if word in model.wv]
    if vectors:
        return sum(vectors) / len(vectors)
    else:
        return None

df['Vector'] = df['text'].apply(vectorize_text)

# Division des données
X_train, X_test, y_train, y_test = train_test_split(df['Vector'].tolist(), df['tag'], test_size=0.2, random_state=42)

# Construction d'un classificateur LogisticRegression
classifier = LogisticRegression(max_iter=1000)

# Fit du modèle sur les données d'entraînement
classifier.fit(X_train, y_train)

# Prédiction sur les données de test
y_pred = classifier.predict(X_test)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (51776,) + inhomogeneous part.

In [3]:
# Évaluation du modèle
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

NameError: name 'y_pred' is not defined